In [3]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def clean_text(text, n_skip_lines=15):
    text_clean = "\n".join(text.split("\n")[n_skip_lines:])
    return text_clean


In [4]:
# params
openai_model = "gpt-3.5-turbo"
data_path = './data/emails_train_sample.csv'
nrows = 1000
#nrows = None

In [5]:
df = pd.read_csv(data_path, nrows=nrows)
print(df.shape)

(1000, 2)


In [15]:
display(df.head())

,file,message
0,ruscitti-k/inbox/4.,Message-ID: <8472651.1075845282216.JavaMail.ev...
1,lenhart-m/deleted_items/167.,Message-ID: <18748638.1075854964503.JavaMail.e...
2,mann-k/all_documents/702.,Message-ID: <2912554.1075845592770.JavaMail.ev...
3,delainey-d/_sent_mail/344.,Message-ID: <33094249.1075854487085.JavaMail.e...
4,jones-t/sent/1156.,Message-ID: <5799064.1075847178612.JavaMail.ev...


In [8]:
df_sample = df.sample(1)
text = df_sample.message.values[0]
print(text)

Message-ID: <9528079.1075840796899.JavaMail.evans@thyme>
Date: Wed, 10 Oct 2001 09:23:30 -0700 (PDT)
From: j.kaminski@enron.com
Subject: Patel
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Kaminski, Vince J </O=ENRON/OU=NA/CN=RECIPIENTS/CN=VKAMINS>
X-To: 
X-cc: 
X-bcc: 
X-Folder: \vkamins\Tasks
X-Origin: KAMINSKI-V
X-FileName: vincent kaminski 1-30-02.pst

Keyur Patel
MIT Sloan School of Management, Class of 2002
617-290-3703 (Mobile) * keyur@mit.edu


In [9]:
text_clean = clean_text(text)
print(text_clean)

Keyur Patel
MIT Sloan School of Management, Class of 2002
617-290-3703 (Mobile) * keyur@mit.edu


In [13]:
prompt = f"""
perform PII entity extraction from the below email message(s).

please return the PII related extracted entities in a long format of key-value pairs like

[
    {{"<entity-type>": "<entity-value>"}},
    {{"<entity-type>": "<entity-value>"}},
...
]

```
{text_clean}
```
"""
print(prompt)


perform PII entity extraction from the below email message(s).

please return the PII related extracted entities in a long format of key-value pairs like

[
    {"<entity-type>": "<entity-value>"},
    {"<entity-type>": "<entity-value>"},
...
]

```
Keyur Patel
MIT Sloan School of Management, Class of 2002
617-290-3703 (Mobile) * keyur@mit.edu
```



In [14]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    model=openai_model,
)
chat_completion_message = chat_completion.choices[0].message
print(chat_completion_message.content)

[
    {"Name": "Keyur Patel"},
    {"School": "MIT Sloan School of Management"},
    {"Year of Graduation": "2002"},
    {"Phone Number": "617-290-3703"},
    {"Email": "keyur@mit.edu"}
]
